Members
1. Muhammad Zikri Ramadhan (5025211085)
2. Sandhika Surya (5025211022)


The data for 2013 flight departing from
- Newark Liberty Airport (NWE)
- LaGuardia Airport (LGA)
- John F Kennedy Airport (JFK)

In [ ]:
import pandas as pd

data = pd.read_csv('flights.csv')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
data.head(10)

In [ ]:
data.nunique() # numbers of unique values for each features

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
# Check null on each columns
data.isnull().sum()

In [ ]:
duplicate_rows = data[data.duplicated()]
print("number of duplicate rows: ", duplicate_rows.shape)
data.info()

In [ ]:
# Dropping column id, year, time_hour, and name
# id : id of the record
# year : year of the record. all data is from 2013
# time_hour : departure data in format yyyy-mm-dd hh-mm-ss
# name : full name of the airlines

data.drop(columns=['id', 'year', 'time_hour', 'name'], axis=1, inplace=True)
data.info()

In [ ]:
# the count of each unique values for each categorical values

for col in data.columns:
  if data.dtypes[col] == 'object':
    print(f'{col} : {data[col].value_counts()}\n')

In [ ]:
data.describe().T

In [ ]:
# Creating isWeekend feature
# To know if a flight is on weekend/weekday based on the date

data['isWeekend'] = pd.to_datetime({'year': 2013, 'month': data['month'], 'day': data['day']}).dt.weekday >= 5
data['isWeekend'] = data['isWeekend'].astype(int)

In [ ]:
# Changing hhmm format to minutes from 00:00
# Calculating scheduled air time
# sched_air_time = sched_arr_time - sched_dep_time

def hhmm_to_minutes(time):
    hours = time // 100
    minutes = time % 100
    return hours * 60 + minutes

data['dep_time'] = data['dep_time'].apply(lambda x: hhmm_to_minutes(int(x)) if pd.notna(x) else x)
data['arr_time'] = data['arr_time'].apply(lambda x: hhmm_to_minutes(int(x)) if pd.notna(x) else x)
data['sched_dep_time'] = data['sched_dep_time'].apply(hhmm_to_minutes)
data['sched_arr_time'] = data['sched_arr_time'].apply(hhmm_to_minutes)

data['sched_air_time'] = abs(data['sched_arr_time'] - data['sched_dep_time'])

In [ ]:
# Imputation based on this formula:
# air_time = arr_time - dep_time

def impute_times(row):
    if row['air_time'] is np.nan and pd.notna(row['arr_time']) and pd.notna(row['dep_time']):
        row['air_time'] = row['arr_time'] - row['dep_time']
    elif row['arr_time'] is np.nan and pd.notna(row['air_time']) and pd.notna(row['dep_time']):
        row['arr_time'] = row['air_time'] + row['dep_time']
    elif row['dep_time'] is np.nan and pd.notna(row['arr_time']) and pd.notna(row['air_time']):
        row['dep_time'] = row['arr_time'] - row['air_time']
    return row

data = data.apply(impute_times, axis=1)

# Delete row with more than 1 of those row that's NaN
data = data.dropna(subset=['dep_time', 'arr_time', 'air_time'], thresh=2)

In [ ]:
# Filling remaining null on dep_delay, arr_delay, and air_time
# Filling based on calculations
# dep_delay = dep_time - sched_dep_time
# arr_delay = arr_time - sched_arr_time
# air_time = arr_time - dep_time

data['dep_delay'] = data['dep_time'] - data['sched_dep_time']
data['arr_delay'] = data['arr_time'] - data['sched_arr_time']
data['air_time'] = data['arr_time'] - data['dep_time']
data.info()

In [ ]:
# Categorize flight duration
# Short haul (0 - 180 mins)
# Medium haul (180 - 360 mins)
# Long haul (360 - 720 mins)
# Ultra long haul (> 720 mins)

def categorize_flight(duration):
    if duration < 180:
        return 'Short Flight'
    elif duration < 360:
        return 'Medium Flight'
    elif duration <= 720:
        return 'Long Flight'
    else:
        return 'Ultra Long Flight'

data['flight_category'] = data['sched_air_time'].apply(categorize_flight)

In [ ]:
# Categorize scheduled departure time

def categorize_dep_time(sched_dep):
    if sched_dep < 360:  # 00:00 - 05:59
        return '1'
    elif sched_dep < 720:  # 06:00 - 11:59
        return '2'
    elif sched_dep < 1080:  # 12:00 - 17:59
        return '3'
    else:  # 18:00 - 23:59
        return '4'

data['sched_dep_category'] = data['sched_dep_time'].apply(categorize_dep_time)


In [ ]:
data.head(10)

In [ ]:
data.isnull().sum()

In [ ]:
imputer = SimpleImputer(strategy='most_frequent')

catColumns = data.select_dtypes(include=['object'])
catNan = catColumns.columns[catColumns.isna().any()]
print(catNan) # Check which categorical columns have missing values

for col in catColumns:
  encoder = LabelEncoder()
  data[col] = encoder.fit_transform(data[col].astype(str)) # Encoding categorical values into numerical representation
  if col in catNan:
    data[col] = imputer.fit_transform(data[[col]]) # Filling Nan values with a most frequent value (mode)

data.info()

In [ ]:
data.boxplot(figsize=(12, 10))
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Display the data distribution for each column
plt.figure(figsize=(12, 10))
data.hist(bins=100, figsize=(12, 10), edgecolor='black', layout=(5, 5))

plt.tight_layout()
plt.show()

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for each column
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1

# Keep rows where all values are within the IQR range (no outliers)
data = data[((data >= (Q1 - 1.5 * IQR)) & (data <= (Q3 + 1.5 * IQR))).all(axis=1)]
data.shape

In [ ]:
data.boxplot(figsize=(12, 10))
plt.xticks(rotation=90)
plt.show()

In [ ]:
corr = data.corr()

plt.figure(figsize=(15,10))

plt.xticks(rotation=90)
plt.yticks(rotation=0)
sns.heatmap(corr, annot=True, fmt='.2f', cmap='Blues', linewidths=1)
plt.show()

In [ ]:
data.info()

In [ ]:
data.reset_index()

In [ ]:
# Display the data distribution for each column
plt.figure(figsize=(12, 10))
data.hist(bins=100, figsize=(12, 10), edgecolor='black', layout=(5, 5))

plt.tight_layout()
plt.show()

In [ ]:
# correlation between two columns
sns.scatterplot(data=data, x='sched_air_time', y='flight_category')
plt.show()

In [ ]:
sns.scatterplot(data=data, x='sched_air_time', y='distance')
plt.show()

In [ ]:
# correlation between two columns
sns.scatterplot(data=data, x='sched_air_time', y='distance')
plt.show()

In [ ]:
# Checking unique value from these columns

print(data["origin"].unique())
print(data["dest"].unique())
print(data["carrier"].unique())

In [ ]:
# Creating isWeekend feature
# To know if a flight is on weekend/weekday based on the date

data['isWeekend'] = pd.to_datetime({'year': 2013, 'month': data['month'], 'day': data['day']}).dt.weekday >= 5
data['isWeekend'] = data['isWeekend'].astype(int)

In [ ]:
# Changing hhmm format to minutes from 00:00
# Calculating scheduled air time
# sched_air_time = sched_arr_time - sched_dep_time

def hhmm_to_minutes(time):
    hours = time // 100
    minutes = time % 100
    return hours * 60 + minutes

data['dep_time'] = data['dep_time'].apply(lambda x: hhmm_to_minutes(int(x)) if pd.notna(x) else x)
data['arr_time'] = data['arr_time'].apply(lambda x: hhmm_to_minutes(int(x)) if pd.notna(x) else x)
data['sched_dep_time'] = data['sched_dep_time'].apply(hhmm_to_minutes)
data['sched_arr_time'] = data['sched_arr_time'].apply(hhmm_to_minutes)

data['sched_air_time'] = abs(data['sched_arr_time'] - data['sched_dep_time'])

In [ ]:
# Imputation based on this formula:
# air_time = arr_time - dep_time

def impute_times(row):
    if row['air_time'] is np.nan and pd.notna(row['arr_time']) and pd.notna(row['dep_time']):
        row['air_time'] = row['arr_time'] - row['dep_time']
    elif row['arr_time'] is np.nan and pd.notna(row['air_time']) and pd.notna(row['dep_time']):
        row['arr_time'] = row['air_time'] + row['dep_time']
    elif row['dep_time'] is np.nan and pd.notna(row['arr_time']) and pd.notna(row['air_time']):
        row['dep_time'] = row['arr_time'] - row['air_time']
    return row

data = data.apply(impute_times, axis=1)

# Delete row with more than 1 of those row that's NaN
data = data.dropna(subset=['dep_time', 'arr_time', 'air_time'], thresh=2)

In [ ]:
# Filling remaining null on dep_delay, arr_delay, and air_time
# Filling based on calculations
# dep_delay = dep_time - sched_dep_time
# arr_delay = arr_time - sched_arr_time
# air_time = arr_time - dep_time

data['dep_delay'] = data['dep_time'] - data['sched_dep_time']
data['arr_delay'] = data['arr_time'] - data['sched_arr_time']
data['air_time'] = data['arr_time'] - data['dep_time']
data.info()

In [ ]:
# Categorize flight duration
# Short haul (0 - 180 mins)
# Medium haul (180 - 360 mins)
# Long haul (360 - 720 mins)
# Ultra long haul (> 720 mins)

def categorize_flight(duration):
    if duration < 180:
        return 'Short Flight'
    elif duration < 360:
        return 'Medium Flight'
    elif duration <= 720:
        return 'Long Flight'
    else:
        return 'Ultra Long Flight'

data['flight_category'] = data['sched_air_time'].apply(categorize_flight)